In [ ]:
import os
import sys
import joblib
sys.path.insert(1, '/mnt/d/PowerTAC/Python/python_codes/helper') 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from statistics import mean
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from read_mongo_collection import HelperToReadMongo

sns.set_theme(style="darkgrid")

# See complete data
# pd.set_option('max_columns', None)
# pd.set_option("max_rows", None)
# np.set_printoptions(threshold=np.inf)

In [ ]:
helper_mongo = HelperToReadMongo()

In [ ]:
# Specify names of all the databases here
database = 'PowerTAC2022_ResidentialEV'
collection = 'Residential EV Usage'

# Specify the storage path along with directory
storage_path = '/mnt/d/PowerTAC/PowerTAC2022/experiments/residentialEV/usage_plots/'
os.makedirs(storage_path, exist_ok=True) 

In [ ]:
db = helper_mongo.query_to_mongo(database, collection, server_ip='<IP>', ssh_username='<username>', ssh_password='<password>', remote=True)  

In [ ]:
games = db['Game_Name'].unique()

for game in games:
    
    db1 = db[db['Game_Name']==game]
    db1['Usage'] *= -1
    fig = px.line(db1, x='Timeslot', y='Usage')
    
    fig.update_yaxes(title_text="Usage (KWh)")
    fig.update_xaxes(title_text="Timeslot")

    fig.update_layout(height=750, width=1500, title_text=game, template='ggplot2')

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)

    fig.write_html(storage_path + "/" + game + "_overall.html")

In [ ]:
games = db['Game_Name'].unique()

for game in games:
    
    db1 = db[db['Game_Name']==game]
    db1['Usage'] *= -1
    db2 = db1.groupby('Day_of_Week').mean()
    db3 = db1.groupby('Day_of_Week').std()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=db2['Usage'], mode='lines', name='mean'))
    fig.add_trace(go.Scatter(y=db2['Usage'] - db3['Usage'], mode='lines', name='mean-std'))
    fig.add_trace(go.Scatter(y=db2['Usage'] + db3['Usage'], mode='lines', name='mean+std'))
    
#     fig = px.line(db1['Usage'])  
    
    fig.update_yaxes(title_text="Usage (KWh)")
    fig.update_xaxes(title_text="Day of Week")

    fig.update_layout(height=750, width=1500, title_text=game, template='ggplot2')

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)

    fig.write_html(storage_path + "/Weekly/" + game + "_weekly.html")

In [ ]:
games = db['Game_Name'].unique()

for game in games:
    
    db1 = db[db['Game_Name']==game]
    db1['Usage'] *= -1
    db2 = db1.groupby('Hour_of_Day').mean()
    db3 = db1.groupby('Hour_of_Day').std()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=db2['Usage'], mode='lines', name='mean'))
    fig.add_trace(go.Scatter(y=db2['Usage'] - db3['Usage'], mode='lines', name='mean-std'))
    fig.add_trace(go.Scatter(y=db2['Usage'] + db3['Usage'], mode='lines', name='mean+std'))
        
    fig.update_yaxes(title_text="Usage (KWh)")
    fig.update_xaxes(title_text="Hour of Day")

    fig.update_layout(height=750, width=1500, title_text=game, template='ggplot2')

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)

    fig.write_html(storage_path + "/Hourly/" + game + "_hourly.html")